## Análise de Correlação

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive')

In [ ]:
import os
import random
from PIL import Image
import numpy as np

In [ ]:
img_dim = 64

def create_fixed_permutation(shape, seed=42):
    np.random.seed(seed)
    indices = np.arange(np.prod(shape))
    np.random.shuffle(indices)
    return indices.reshape(shape).flatten()

def apply_permutation(image, permutation):
    arr = np.array(image)
    flat_arr = arr.flatten()
    shuffled_arr = flat_arr[permutation]
    return Image.fromarray(shuffled_arr.reshape(arr.shape))

def process_images_with_fixed_permutation(root_dir, seed=42):
    """
    Processa todas as imagens em um diretório, embaralhando os pixels
    de acordo com uma permutação fixa.
    """
    first_image_path = None
    target_size = (img_dim, img_dim)
    processadas = 0

    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    img = img.convert('L')  # L para grayscale
                    img = img.resize(target_size, Image.LANCZOS)
                    if first_image_path is None:
                        first_image_path = file_path
                        permutation = create_fixed_permutation(np.array(img).shape, seed)

                    shuffled_img = apply_permutation(img, permutation)
                    # Salvar remapeamento
                    shuffled_img.save(file_path)
                    processadas += 1
                    if processadas % 10000 == 0:
                        print(f"Processadas {processadas} imagens")
            except Exception as e:
                print(f"Falha ao processar {file_path}: {e}")

In [ ]:
#Carregue os dados PKLotSegmented.tar.gz para seu Google Drive antes de executar
#Ajuste o path para apontar para o local do arquivo

!rm -rf /content/datasets
!mkdir /content/datasets
!rm -rf /content/datasets_shuffle
!mkdir /content/datasets_shuffle

!tar -xf "/content/drive/MyDrive/PKLotSegmented.tar.gz" -C "/content/datasets"
#Ou, se o .tar.gz já estiver carregado no seu ambiente:
!tar -xf "/content/PKLotSegmented.tar.gz" -C "/content/datasets"

!mv /content/datasets/Treino-UFPR04 /content/datasets/treino
!mv /content/datasets/Validação-UFPR05 /content/datasets/validacao
!mv /content/datasets/Teste-PUC /content/datasets/teste

!cp -r /content/datasets/ /content/datasets_shuffle

!mv /content/datasets_shuffle/datasets/* /content/datasets_shuffle/
!rm -r /content/datasets_shuffle/datasets/

process_images_with_fixed_permutation('/content/datasets_shuffle/')

In [ ]:
# Para ver as imagens com shuffle de pixels
from IPython.display import Image as image_display
from IPython.display import display

display(image_display('/content/datasets/treino/Occupied/2012-12-07_16_42_25#001.jpg'))
display(image_display('/content/datasets_shuffle/treino/Occupied/2012-12-07_16_42_25#001.jpg'))

In [ ]:
import math
import copy

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [ ]:
# Dataloaders


#ATENÇÃO: Execute primeiro os experimentos com /content/datasets
# Depois, entenda como são as imagens no diretório /content/datasets/shuffle, e então execute os experimentos usando esse diretório
data_dir = "/content/datasets"
#data_dir = "/content/datasets_shuffle"

#As imagens tratadas serão em escala de cinza para diminuir o tamanho das redes
data_transforms = {
    "treino": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "validacao": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "teste": transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
}

sets = ["treino", "validacao", "teste"]

print("Criando dataloaders para diretório", data_dir)

image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
                  for x in sets}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
               for x in sets}

dataset_sizes = {x: len(image_datasets[x]) for x in sets}
class_names = image_datasets['treino'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Conectado em um ambiente com", device)

In [ ]:
INPUT_SHAPE = (1, img_dim, img_dim)
NUM_CLASSES = 2

#Definição de uma CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=INPUT_SHAPE[0], out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, NUM_CLASSES)

    def convs(self, x):
        x = f.relu(self.conv1(x))
        x = self.pool1(x)
        x = f.relu(self.conv2(x))
        x = self.pool2(x)
        x = f.relu(self.conv3(x))
        x = self.pool3(x)

        return x


    def forward(self, x):
        x = self.convs(x)
        x = self.flatten(x)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)

        return x

#Definição de um MLP convencional de 3 camadas ocultas
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], output_size)
        self.relu = nn.ReLU()
        self.dropout_input = nn.Dropout(0.1)
        self.dropout_hidden = nn.Dropout(0.2)

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout_input(x)
        x = self.relu(self.fc1(x))
        x = self.dropout_hidden(x)
        x = self.relu(self.fc2(x))
        x = self.dropout_hidden(x)
        x = self.relu(self.fc3(x))
        x = self.dropout_hidden(x)
        x = self.fc4(x)
        return x

In [ ]:
# Função para treinar o modelo
def train_model(model, criterion, optimizer, epochs=3):
    best_loss = math.inf

    for epoch in range(epochs):
        print(f"Época {epoch+1}/{epochs}")
        print('-' * 10)

        for phase in ["treino", "validacao"]:
            if phase == "treino":
                model.train()  # Colocar o modelo em modo de treino
            else:
                model.eval()   # Colocar o modelo em modo de avaliação

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "treino"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "treino":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return

In [ ]:
print("Testes para CNN")

#Treinamento
#model_cnn = CNN()

model_cnn = models.mobilenet_v3_small(pretrained=True)

model_cnn.features[0][0] = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model_cnn.classifier[3] = nn.Linear(model_cnn.classifier[3].in_features, 2)

for param in model_cnn.parameters():
    param.requires_grad = True

model_cnn = model_cnn.to(device)

# Função de loss e o otimizador
criterion = nn.CrossEntropyLoss()
#optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.002)
optimizer_cnn = optim.SGD(model_cnn.parameters(), lr=0.002, momentum=0.9)

# Treinar o modelo
print("Treinando")
train_model(model_cnn, criterion, optimizer_cnn, epochs=5)

# Avaliar o modelo no conjunto de teste
print("Avaliando CNN no conjunto de testes")
model_cnn.eval()
test_corrects = 0

for inputs, labels in dataloaders['teste']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_cnn(inputs)
    _, preds = torch.max(outputs, 1)
    test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / dataset_sizes['teste']
print(f'Test Acc: {test_acc:.4f}')


print("Testes para MLP")

input_size = img_dim * img_dim
hidden_sizes = [512, 256, 128]
output_size = len(class_names)

model_mlp = MLP(input_size, hidden_sizes, output_size)
model_mlp = model_mlp.to(device)

# otimizador
#optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=0.001)
optimizer_mlp = optim.SGD(model_mlp.parameters(), lr=0.005, momentum=0.9)

# Treinar o modelo
print("Treinando")
train_model(model_mlp, criterion, optimizer_mlp, epochs=5)

# Avaliar o modelo no conjunto de teste
print("Avaliando MLP no conjunto de testes")
model_mlp.eval()
test_corrects = 0

for inputs, labels in dataloaders['teste']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_mlp(inputs)
    _, preds = torch.max(outputs, 1)
    test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / dataset_sizes['teste']
print(f'Test Acc: {test_acc:.4f}')